In [8]:
### BUSINESS CHALLENGE:

# Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

# We will be provided a company name and their primary website.

In [9]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [10]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [11]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [13]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [44]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [50]:
import ollama
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def get_links(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [51]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/nari-labs/Dia-1.6B',
 '/sand-ai/MAGI-1',
 '/microsoft/bitnet-b1.58-2B-4T',
 '/ostris/Flex.2-preview',
 '/THUDM/GLM-4-32B-0414',
 '/models',
 '/spaces/nari-labs/Dia-1.6B',
 '/spaces/enzostvs/deepsite',
 '/spaces/InstantX/InstantCharacter',
 '/spaces/bytedance-research/UNO-FLUX',
 '/spaces/nvidia/describe-anything-model-demo',
 '/spaces',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/Anthropic/values-in-the-wild',
 '/datasets/OpenGVLab/InternVL-Data',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/zwhe99/DeepMath-103K',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/doc

In [52]:
### Make the brochure!

In [53]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [66]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [67]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [58]:
get_brochure_user_prompt("Massimo Dutti", "https://www.massimodutti.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.massimodutti.com/en/about'}, {'type': 'careers page', 'url': 'https://www.massimodutti.com/en/careers'}, {'type': 'contact us page', 'url': 'https://www.massimodutti.com/en/contact'}, {'type': 'home/featured designs page', 'url': 'https://www.massimodutti.com/featured-designs'}]}


'You are looking at a company called: Massimo Dutti\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\n\xa0\nWebpage Contents:\n\n\n\n\nabout page\nWebpage Title:\nMassimo Dutti\nWebpage Contents:\nPage does not exist.\nMaybe you got a broken link, or maybe you made a misprint in the address bar.\nGO TO HOME PAGE\n\n\n\ncareers page\nWebpage Title:\nMassimo Dutti\nWebpage Contents:\nPage does not exist.\nMaybe you got a broken link, or maybe you made a misprint in the address bar.\nGO TO HOME PAGE\n\n\n\ncontact us page\nWebpage Title:\nMassimo Dutti\nWebpage Contents:\nPage does not exist.\nMaybe you got a broken link, or maybe you made a misprint in the address bar.\nGO TO HOME PAGE\n\n\n\nhome/featured designs page\nWebpage Title:\nMassimo Dutti\nWebpage Contents:\nPage does not exist.\nMaybe you got a broken link, or maybe you made a misprint in the address b

In [68]:
def create_brochure(company_name, url):
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [60]:
create_brochure("Massimo Dutti", "https://www.massimodutti.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://www.massimodutti.com'}, {'type': 'about page', 'url': 'https://www.massimodutti.com/our-story'}, {'type': 'products page', 'url': 'https://www.massimodutti.com/products'}, {'type': 'collections page', 'url': 'https://www.massimodutti.com/collections/all'}, {'type': 'stores/ physical store location', 'url': 'https://www.massimodutti.com/stores'}, {'type': 'contact us page', 'url': 'https://www.massimodutti.com/contact'}]}


**Massimo Dutti Brochure**
==========================

**About Us**
------------

Welcome to Massimo Dutti, a leading international fashion brand known for its sophisticated and elegant designs. Our mission is to provide high-quality clothing and accessories that cater to the needs of discerning individuals who value style, comfort, and exclusivity.

**Our Story**
-------------

Founded in 1985 by Martino Dutti, our company has grown into a global powerhouse with over 300 stores in more than 60 countries. We are proud of our heritage and continue to innovate while staying true to our core values.

**Company Culture**
-----------------

At Massimo Dutti, we value:

*   **Excellence**: We strive for perfection in every aspect of our operations.
*   **Respect**: We treat all employees with respect and dignity.
*   **Innovation**: We encourage creativity and new ideas to stay ahead in the industry.
*   **Sustainability**: We aim to reduce our environmental impact through sustainable practices.

**Our Customers**
----------------

We cater to a wide range of stylish individuals who appreciate our sophisticated designs. Our customers include fashion-conscious men and women from all walks of life, from professionals to socialites.

**Careers & Join Us**
-------------------

Are you passionate about fashion and ready for a new challenge? We're looking for talented individuals to join our team. With opportunities in design, marketing, sales, and more, we offer a dynamic work environment that fosters growth and development.

Visit our website or store today to experience the world of Massimo Dutti.

**Stores & Locations**
---------------------

Find us in over 300 stores worldwide:

[Insert list of countries and store locations]

**Contact Us**
-------------

Reach out to us for more information on our products, careers, or store locations:

[Insert contact email and phone number]

Stay fashionable with Massimo Dutti!

In [61]:
### A minor improvement

In [69]:
def stream_brochure(company_name, url):
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [63]:
stream_brochure("Massimo Dutti", "https://www.massimodutti.com")

Found links: {'links': [{'type': 'home page', 'url': 'https://www.massimodutti.com'}, {'type': 'about us page', 'url': 'https://www.massimodutti.com/About-Us/'}, {'type': 'products page', 'url': 'https://www.massimodutti.com/Products/'}, {'type': 'careers/jobs page', 'url': 'https://www.massimodutti.com/Careers/'}, {'type': 'locations/page', 'url': 'https://www.massimodutti.com/Locations/'}, {'type': 'contact us page', 'url': 'https://www.massimodutti.com/Contact-Us.aspx'}]}


# Massimo Dutti Brochure

## About Us

We are Massimo Dutti, the fashion powerhouse that brings style and sophistication to our customers. Our story began with a vision to revive Italian craftsmanship and elegance on a global scale.

At Massimo Dutti, we are passionate about creating apparel that makes you feel confident, stylish, and unique. From iconic pieces to statement accessories, each item in our collection is designed to tell a story of understated luxury.

## Company Culture

* **Elegance**: We believe that beauty lies in the details.
* **Quality**: Our clothes are crafted with precision and attention to detail.
* **Innovation**: We stay on top of the latest trends while staying true to our heritage.
* **Sustainability**: We strive to reduce our environmental impact through sustainable practices.

## Customers

Our customers are fashionistas who value style, quality, and exclusivity. They appreciate our dedication to delivering impeccable apparel that enhances their personal style and makes them feel confident in any setting.

Join the Massimo Dutti community of stylish individuals who demand the best from themselves and from their clothing.

## Careers/Jobs

At Massimo Dutti, we are committed to creating a workplace environment where creativity, innovation, and teamwork come together. Our talented team members empower each other to excel in their fields while having fun and growing as professionals.

If you share our passion for fashion, leadership, and growth, join us on this exciting journey!

### Locations

We operate in multiple locations around the world, with showrooms in major cities where our stylish customers reside. Follow the link to find your nearest Massimo Dutti store or online platform.

[Location Page](insert link)

## Contact Us

Want to get in touch with us? We're here to help any questions about our products, collections, or careers. Fill out the contact form and we'll answer you as soon as possible.

[Contact Us Page](insert link)

In [70]:
# Try changing the system prompt to the humorous version when you make the Brochure for:

stream_brochure("Massimo Dutti", "https://www.massimodutti.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.massimodutti.com/en/about/'}, {'type': 'our story', 'url': 'https://www.massimodutti.com/en/blood-and-silk-story/'}, {'type': 'careers page', 'url': 'https://www.massimodutti.com/en/career- opportunities/'}, {'type': 'shop our collections', 'url': 'https://www.massimodutti.com/en/shop'}]}


# Welcome to Massimo Dutti: Where Fashion Meets Passion

[Cover Image: A stylish and elegant image of Massimo Dutti's logo or a beautiful outfit from their collection]

## About Us

Massimo Dutti is more than just a fashion brand - it's a promise of quality, style, and passion. Our story began in 2001 with the vision to create clothing that combines traditional craftsmanship with modern sensibilities.

But what really sets us apart? We take pride in providing you with exceptional service, tailored recommendations from our expert consultants, and a retail experience that's nothing short of unforgettable.

**Our Values**

* **Fashion for Life**: We believe that fashion is not just about clothes, but about living life to the fullest.
* **Passion & Expertise**: Our team of style enthusiasts and craftsmen dedicate themselves to creating clothing that makes you feel confident and beautiful.
* **Sustainable Style**: At Massimo Dutti, we're committed to reducing our environmental footprint and promoting sustainable fashion practices.

## Meet Our Team

Our talented team is made up of passionate individuals who share a common passion for fashion. Whether you're joining as a consultant, sales associate, or in-house designer, you'll be part of a dynamic community that's dedicated to making style happen.

**What We Offer**

* Competitive salaries and benefits packages
* Opportunities for career growth and professional development
* Collaborative work environment with like-minded individuals

## Shop Our Collections

Indulge in our stunning collections, crafted with meticulous attention to detail and designed to make you feel your best. From elegant evening wear to everyday essentials, we've got you covered.

[Image: A showcase of Massimo Dutti's latest collection]

Ready to experience the ultimate shopping destination? Visit us today!

[Call-to-Action Button: Shop Now / Learn More]